In [1]:
from osgeo import ogr
from osgeo import gdal
import bisect
import random
from shapely.geometry import Polygon, Point
from shapely import wkt
from shapely.geometry import shape
from shapely import speedups
speedups.disable()
import math
import pandas as pd
import sys
import datetime
from pyproj import Proj, Transformer
import pyproj

import cdsapi
import xarray as xr
from urllib.request import urlopen

import matplotlib.pyplot as plt
import geopandas as gpd

from pyproj import Proj
from shapely.ops import transform
from shapely.geometry import shape

In [2]:
Directorio = '/Users/daniel/Desktop/Arbeit/IMPORTANTE/Trabajo CMPC/2021-06-27/'
#
zonasUrbanas = 'Areas urbanas/areas_urbanas.shp'
#
fundosFile= 'fundos 4326/fundos 4326.shp'
#
usoSuelo = 'uso suelo 4326/uso suelo 4326.shp'

In [3]:
utm = pyproj.CRS('EPSG:32719')
wgs84 = pyproj.CRS('EPSG:4326')
utm_32518 = pyproj.CRS('EPSG:32518')

project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
project_2 = pyproj.Transformer.from_crs(utm_32518, utm, always_xy=True).transform
project_3 = pyproj.Transformer.from_crs(utm_32518, wgs84, always_xy=True).transform
project_4 = pyproj.Transformer.from_crs(utm, wgs84, always_xy=True).transform
project_5 = pyproj.Transformer.from_crs(utm, utm_32518, always_xy=True).transform
project_6 = pyproj.Transformer.from_crs(wgs84, utm_32518, always_xy=True).transform

In [4]:
def readFundos():
    fundos = gpd.read_file(fundosFile)
    fundos = fundos.loc[fundos['geometry'].is_valid, :]
    fundos['geometry'] = [transform(project_2, fundo) 
                                     for fundo in fundos.geometry]
    return fundos

In [5]:
def readPoligonosCiudades():
    Urbanos = gpd.read_file(zonasUrbanas)
    Urbanos = Urbanos.loc[Urbanos['geometry'].is_valid, :]
    return Urbanos

In [6]:
def readUsoSuelo():
    Suelo = gpd.read_file(usoSuelo)
    Suelo = Suelo.loc[Suelo['geometry'].is_valid, :]
    Suelo['geometry'] = [transform(project_2, suelo) 
                                     for suelo in Suelo.geometry]
    return Suelo

In [9]:
fundosShp = readFundos()
ciudadShp = readPoligonosCiudades()

In [10]:
sueloShp = readUsoSuelo()

DriverError: uso suelo 4326/uso suelo 4326.shp: No such file or directory

### Probando con un punto

In [12]:
point_transform_32718 =transform(project, point)

NameError: name 'point' is not defined

In [13]:
fundo_min_dist = 10000000
fundo_mas_cercano = None
nombre = ''
identificador = -1

for index, item in fundosShp.iterrows():    
    fundo = item.geometry
    dist = fundo.distance(point_transform_32718)
    if dist < fundo_min_dist:
        fundo_min_dist = dist
        fundo_mas_cercano = fundo
        nombre = item.nombre
        identificador = item.id

NameError: name 'point_transform_32718' is not defined

In [14]:
fundo_min_dist, fundo_mas_cercano, nombre, identificador

(10000000, None, '', -1)

In [15]:
p1 = gpd.GeoSeries(fundo_mas_cercano)
p2 = gpd.GeoSeries(point_transform_32718)

fig, ax = plt.subplots(figsize = (20,16))
p1.plot(ax=ax)
p2.plot(color = 'red', ax=ax)
plt.show()

NameError: name 'point_transform_32718' is not defined

### Prueba con API OSM

In [11]:
import requests

url = 'http://router.project-osrm.org/table/v1/driving/13.388860,52.517037;13.397634,52.529407'
res = requests.get(url)

if res.status_code == 200:
    print(res.text)

{"code":"Ok","durations":[[0,251.6],[258.2,0]],"sources":[{"hint":"ZisCjdYrAo0YAAAABQAAAAAAAAAgAAAASjFaQdLNK0AAAAAAsPePQQwAAAADAAAAAAAAABAAAACC5QAA_kvMAKlYIQM8TMwArVghAwAA7wq7dmj-","distance":4.231666,"location":[13.388798,52.517033],"name":"Friedrichstraße"},{"hint":"QlYwgJ8u_4wGAAAACgAAAAAAAAB2AAAAW7-PQOKcyEAAAAAApq6DQgYAAAAKAAAAAAAAAHYAAACC5QAAf27MABiJIQOCbswA_4ghAwAAXwW7dmj-","distance":2.789393,"location":[13.397631,52.529432],"name":"Torstraße"}],"destinations":[{"hint":"ZisCjdYrAo0YAAAABQAAAAAAAAAgAAAASjFaQdLNK0AAAAAAsPePQQwAAAADAAAAAAAAABAAAACC5QAA_kvMAKlYIQM8TMwArVghAwAA7wq7dmj-","distance":4.231666,"location":[13.388798,52.517033],"name":"Friedrichstraße"},{"hint":"QlYwgJ8u_4wGAAAACgAAAAAAAAB2AAAAW7-PQOKcyEAAAAAApq6DQgYAAAAKAAAAAAAAAHYAAACC5QAAf27MABiJIQOCbswA_4ghAwAAXwW7dmj-","distance":2.789393,"location":[13.397631,52.529432],"name":"Torstraße"}]}


In [16]:
url = 'http://router.project-osrm.org/route/v1/driving/13.388860,52.517037;13.397634,52.529407?overview=false'
res = requests.get(url)

if res.status_code == 200:
    print(res.text)

{"code":"Ok","waypoints":[{"hint":"ZisCjdYrAo0YAAAABQAAAAAAAAAgAAAASjFaQdLNK0AAAAAAsPePQQwAAAADAAAAAAAAABAAAACC5QAA_kvMAKlYIQM8TMwArVghAwAA7wq7dmj-","distance":4.231666,"location":[13.388798,52.517033],"name":"Friedrichstraße"},{"hint":"QlYwgJ8u_4wGAAAACgAAAAAAAAB2AAAAW7-PQOKcyEAAAAAApq6DQgYAAAAKAAAAAAAAAHYAAACC5QAAf27MABiJIQOCbswA_4ghAwAAXwW7dmj-","distance":2.789393,"location":[13.397631,52.529432],"name":"Torstraße"}],"routes":[{"legs":[{"steps":[],"weight":252.8,"distance":1884.7,"summary":"","duration":251.6}],"weight_name":"routability","weight":252.8,"distance":1884.7,"duration":251.6}]}


In [17]:
if res.json()['code']=='Ok':
    distancia = res.json()['routes'][0]['distance'] #en metros
    duracion = res.json()['routes'][0]['duration'] #en segundos
print(distancia,duracion)

1884.7 251.6


In [18]:
url = 'http://router.project-osrm.org/route/v1/driving/-71.83085668643338,-37.15838090132603;-71.950,-37.24?overview=false'
res = requests.get(url)

if res.status_code == 200:
    print(res.text)

{"code":"Ok","waypoints":[{"hint":"kQvRgpQL0YIAAAAAfQAAAAAAAAC-FwAAAAAAAEhHC0IAAAAA2VTTRAAAAAB9AAAAAAAAAL4XAABrVAAAIbi3-93YyP238rf7EwLJ_QAAPxCZhXaO","distance":1773.401153,"location":[-71.845855,-37.168931],"name":""},{"hint":"Kw_RgjkP0YJNAAAASwAAAAAAAAAdAAAAv6BWQjhPT0IAAAAAT3yjQU0AAABLAAAAAAAAAB0AAABrVAAAiSC2-2PDx_1QIbb7QMPH_QAAzwCZhXaO","distance":18.083517,"location":[-71.950199,-37.239965],"name":"Emilia Viveros"}],"routes":[{"legs":[{"steps":[],"weight":3122,"distance":14548.2,"summary":"","duration":3122}],"weight_name":"routability","weight":3122,"distance":14548.2,"duration":3122}]}


In [19]:
url = 'http://router.project-osrm.org/route/v1/driving/-71.83085668643338,-37.15838090132603;-71.950,-37.24?overview=false'
res = requests.get(url)

if res.status_code == 200:
    print(res.text)

{"code":"Ok","waypoints":[{"hint":"kQvRgpQL0YIAAAAAfQAAAAAAAAC-FwAAAAAAAEhHC0IAAAAA2VTTRAAAAAB9AAAAAAAAAL4XAABrVAAAIbi3-93YyP238rf7EwLJ_QAAPxCZhXaO","distance":1773.401153,"location":[-71.845855,-37.168931],"name":""},{"hint":"Kw_RgjkP0YJNAAAASwAAAAAAAAAdAAAAv6BWQjhPT0IAAAAAT3yjQU0AAABLAAAAAAAAAB0AAABrVAAAiSC2-2PDx_1QIbb7QMPH_QAAzwCZhXaO","distance":18.083517,"location":[-71.950199,-37.239965],"name":"Emilia Viveros"}],"routes":[{"legs":[{"steps":[],"weight":3122,"distance":14548.2,"summary":"","duration":3122}],"weight_name":"routability","weight":3122,"distance":14548.2,"duration":3122}]}


### Importar DataSet y crear instancias

In [5]:
DataSet = 'dataset.csv'
Recursos_aereos = 'recursos_aereos.csv'
Recursos_combate = 'recursos_combate.csv'

In [6]:
import pandas as pd

DataSetDF = pd.read_csv(DataSet)
DataSetDF.head(5)

,Unnamed: 0,pendiente,VPL,coords,WGS84(GPS),date,time,H (%),T (C),V (m/s)
0,0,6.66667,0.005429,"(781410.8857684857, 5882854.141089097)","(-71.83108423317948, -37.15838297117608)",2017-02-14,17:00,71.8215,17.91930,3.33461
1,1,13.33330,0.001672,"(670918.3050503387, 5841153.4465974495)","(-73.06484307489475, -37.56079948860041)",2015-12-30,20:00,64.6337,14.60420,3.22632
2,2,-43.33330,0.005973,"(715599.1743549515, 5727723.4737017)","(-72.52515166055817, -38.57280701983519)",2019-03-18,10:00,90.5010,2.85562,9.23614
3,3,37.77780,0.001325,"(705019.3597288682, 5658491.045153752)","(-72.62582955383753, -39.19866930544828)",2017-01-14,08:00,16.8809,13.16230,8.31740
4,4,-4.44440,0.009777,"(667973.0952557751, 5841184.090236984)","(-73.09817912445067, -37.56106520866647)",2018-01-07,21:00,25.1265,22.06340,4.96412


In [7]:
Recursos_combatedf = pd.read_csv(Recursos_combate,index_col=False)
Recursos_combatedf.head()

,Clave,Base,Subgerencia,Área Patrimonio,EESS,Geo Ref,Geográfica,Dotación,F. de Combate,UTM (X) epsg:32518,UTM (Y) epsg:32518,Geo Lat.,Geo Lon.,Región,Comuna,Tipo
0,315,Cañete,Norte,Cañete,Serfonac,HO 1542 D,Cañete,14,10,641026.36,5814827.40,"37º 48' 11""","73º 23' 53""",Biobio,Cañete,TERRESTRE
1,711,Cholchol,Sur,Carahue,Serfonac,IS 2392 C,4.26 km SE Chol Chol,14,10,691960.24,5722992.44,"38° 37' 14""","72° 47' 41""",Araucania,Cholchol,TERRESTRE
2,713,Cholchol,Sur,Carahue,Serfonac,IS 2392 C,4.26 km SE Chol Chol,6,4,691960.24,5722992.44,"38° 37' 14""","72° 47' 41""",Araucania,Cholchol,CISTERNA
3,414,Cholguan,Norte,Los Angeles,Serfonac,GP 8360 C,6.4 km SW Yungay,14,10,759953.34,5882654.51,"37º 09' 59""","72º 04' 20""",Ñuble,Yungay,TERRESTRE
4,316,Contulmo,Norte,Cañete,Serfonac,HY 9156 A,Contulmo,14,10,656287.37,5791518.34,"38 °00' 37""","73° 13' 10""",Biobio,Contulmo,TERRESTRE


In [8]:
Recursos_aereosdf = pd.read_csv(Recursos_aereos,index_col=False)
Recursos_aereosdf.head(5)

,Clave,Base,Área Patrimonio,EESS,Geo Ref,Geográfica,UTM (X) epsg:32518,UTM (Y) epsg:32518,Geo Lat.,Geo Lon.,Modelo Aeronave,Velocidad Crucero (Km/h),Autonomía de Vuelo,Capacidad de Bombardeo (lt.),Región,Comuna,Tipo
0,Z-5,Angol,Nacimiento,Cerda,HR 1504 C,Aeródromo Angol,703678.66,5814254.95,"37º 47' 46""","72º 41' 11""",Air Tractor 802,259,NaN,3000,IX- Araucanía,Angol,CISTERNAAEREA
1,Z-6,Angol,Nacimiento,Cerda,HR 1504 C,Aeródromo Angol,703678.66,5814254.95,"37º 47' 46""","72º 41' 11""",Air Tractor 802,259,NaN,3000,IX- Araucanía,Angol,CISTERNAAEREA
2,Z-9,Angol,Nacimiento,Cerda,HR 1504 C,Aeródromo Angol,703678.66,5814254.95,"37º 47' 46""","72º 41' 11""",Air Tractor 802,259,NaN,3000,IX- Araucanía,Angol,CISTERNAAEREA
3,Z-4,Cauquenes,Talca,Indef,DW 2742 C,9.5 Km N de Cauquenes,741464.60,6026371.60,"35º 52' 37""","72º 19' 31""",Air Tractor 802,259,NaN,3000,VII- Maule,Cauquenes,CISTERNAAEREA
4,Z-3,María Dolores,Los Angeles,Serfonac,GX 5928 B,Aeródromo Los Angeles,728021.07,5858347.74,"37º 23' 36""","72º 25' 27""",Air Tractor 802,259,NaN,3000,VIII- Biobio,Los Ángeles,CISTERNAAEREA


In [9]:
# Pre-proceso de los recursos antes de crear intancias
# buscar unicos de base
# por base ver si existe la misma cantidad de recursos en cisterna y helicombate si hay más 
# en en recursos aereos

recursosDF = pd.DataFrame(columns=['Clave','UTM (X)','UTM (Y)','Tipo'])
bases = set(Recursos_combatedf.Base.unique().tolist()) | set(Recursos_aereosdf.Base.unique().tolist())

print(bases)

for base in bases:
    combat = Recursos_combatedf[Recursos_combatedf.Base==base]
    aereos = Recursos_aereosdf[Recursos_aereosdf.Base==base]
    #Cisterna
    if len(combat[Recursos_combatedf.Tipo=='CISTERNA']) < len(aereos[Recursos_aereosdf.Tipo=='CISTERNAAEREA']):
        dif = len(aereos[Recursos_aereosdf.Tipo=='CISTERNAAEREA']) - len(combat[Recursos_combatedf.Tipo=='CISTERNA'])
        for i in aereos[Recursos_aereosdf.Tipo=='CISTERNAAEREA'][['Clave','UTM (X) epsg:32518','UTM (Y) epsg:32518','Tipo']].values.tolist()[0:dif]:
            recursosDF.loc[len(recursosDF.index)] = i
    #Helicombate
    if len(combat[Recursos_combatedf.Tipo=='HELICOMBATE']) < len(aereos[Recursos_aereosdf.Tipo=='HELICOMBATE']):
        dif = len(aereos[Recursos_aereosdf.Tipo=='HELICOMBATE']) - len(combat[Recursos_combatedf.Tipo=='HELICOMBATE'])
        for i in aereos[Recursos_aereosdf.Tipo=='HELICOMBATE'][['Clave','UTM (X) epsg:32518','UTM (Y) epsg:32518','Tipo']].values.tolist()[0:dif]:
            recursosDF.loc[len(recursosDF.index)] = [*i[0:3],'CHINOOK']
    #Todos
    #print(combat[['Clave','UTM (X) epsg:32518','UTM (Y) epsg:32518','Tipo']])
    insertar = combat[['Clave','UTM (X) epsg:32518','UTM (Y) epsg:32518','Tipo']].copy()
    insertar.rename(columns = {'UTM (X) epsg:32518':'UTM (X)','UTM (Y) epsg:32518':'UTM (Y)'}, inplace = True)
    recursosDF = recursosDF.append(insertar, ignore_index = True)

del combat, aereos, insertar

{'Talcamavida', 'San Antonio', 'Metropolitana', 'Cañete', 'María Dolores', 'Cauquenes', 'Yumbel', 'Angol', 'Panguilemo', 'Villa La Suerte', 'Tralpenes', 'Tijeral', 'Trilahue', 'Las Cañas', 'San Oscar', 'Contulmo', 'Pan de Azúcar', 'Pencahue', 'Nueva Imperial', 'Cholchol', 'Millantu', 'Pinares', 'Empedrado', 'Victoria', 'Villa Mulchén', 'Cholguan', 'Junquillar '}


<ipython-input-9-4722d9967506>:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if len(combat[Recursos_combatedf.Tipo=='CISTERNA']) < len(aereos[Recursos_aereosdf.Tipo=='CISTERNAAEREA']):
<ipython-input-9-4722d9967506>:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if len(combat[Recursos_combatedf.Tipo=='HELICOMBATE']) < len(aereos[Recursos_aereosdf.Tipo=='HELICOMBATE']):
<ipython-input-9-4722d9967506>:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dif = len(aereos[Recursos_aereosdf.Tipo=='CISTERNAAEREA']) - len(combat[Recursos_combatedf.Tipo=='CISTERNA'])
<ipython-input-9-4722d9967506>:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for i in aereos[Recursos_aereosdf.Tipo=='CISTERNAAEREA'][['Clave','UTM (X) epsg:32518','UTM (Y) epsg:32518','Tipo']].values.tolist()[0:dif]:
<ipython-input-9-4722d9967506>:21: UserWarning: Boolean Series key will be reind

In [10]:
recursosDF

,Clave,UTM (X),UTM (Y),Tipo
0,417,685704.25,5885197.99,HELITRANSPORTADA
1,415,685704.25,5885197.99,TERRESTRE
2,612,726788.53,5797515.41,HELITRANSPORTADA
3,614,726788.53,5797515.41,TERRESTRE
4,611,726788.53,5797515.41,CISTERNA
5,216,911438.40,6269421.94,TERRESTRE
6,315,641026.36,5814827.40,TERRESTRE
7,Z-3,728021.07,5858347.74,CISTERNAAEREA
8,314,728099.06,5858499.85,HELITRANSPORTADA
9,313,728099.06,5858499.85,CISTERNA


In [27]:
#Prueba transformación
for index,item in recursosDF.iterrows():
    item['Clave']
    point = Point(item['UTM (X)'],item['UTM (Y)'])
    point_wgs = transform(project_3,point)
    break
print(point.x,point.y)
print(point_wgs.x,point_wgs.y)

728021.07 5858347.74
-72.42393942587 -37.393609175074175


In [11]:
import math


def distance(origin, destination):
    """
    Calculate the Haversine distance.

    Parameters
    ----------
    origin : tuple of float
        (lat, long)
    destination : tuple of float
        (lat, long)

    Returns
    -------
    distance_in_km : float

    Examples
    --------
    origin = (48.1372, 11.5756)  # Munich
    >>> destination = (52.5186, 13.4083)  # Berlin
    >>> round(distance(origin, destination), 1)
    504.2
    """
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371  # km

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c

    return d

In [12]:
velocidades = {'HELITRANSPORTADA':203, 'CISTERNAAEREA':259, 'CHINOOK':222}

In [13]:
# Se excluyen DE VERDAD los interfaz
recursosDF = recursosDF[recursosDF["Tipo"]!="Interfaz"]

In [41]:
ids = DataSetDF[['date','time']]
unique = {}

for i, row in ids.iterrows():
    tupl = (row['date'],row['time'])
    if tupl not in unique:
        unique[tupl] = [i]
    else:
        unique[tupl].append(i)

counter = dict()
        
for ind in unique.values():
    size = len(ind)
    if size not in counter:
        counter[size] = [ind]
    else:
        counter[size].append(ind)
    
idxPorConcurrencia = [counter[val] for val in sorted(counter.keys())]


In [125]:
### Seleccion de los incendios
numeroIncendios = 3

grupoMuestra = random.randint(numeroIncendios,len(idxPorConcurrencia)-1)
grupoDeFocos = random.randint(0,len(idxPorConcurrencia[grupoMuestra])-1)

mascara = [True]*numeroIncendios + [False]*(grupoMuestra+1-numeroIncendios)
random.shuffle(mascara)

idsIncendios = [item for keep, item in zip(mascara, idxPorConcurrencia[grupoMuestra][grupoDeFocos]) if keep]

incendios = DataSetDF.iloc[idsIncendios, :]
print(incendios)

      Unnamed: 0  pendiente       VPL                                  coords  \
1093        1093   25.55560  0.006516  (764100.3349053301, 5789977.445348817)   
1623        1623   30.00000  0.002596  (705198.0609838904, 5836062.614205524)   
5871        5871    4.44444  0.013174   (699561.5935616073, 5782589.42924405)   

                                    WGS84(GPS)        date   time    H (%)  \
1093  (-71.99229230508983, -37.99943823976221)  2016-01-14  20:00  52.3576   
1623   (-72.6755660110673, -37.59965099716092)  2016-01-14  20:00  54.5716   
5871      (-72.7246134074418, -38.08247391918)  2016-01-14  20:00  56.8467   

        T (C)  V (m/s)  
1093  14.1205  7.86179  
1623  12.1522  5.78450  
5871  13.6859  5.34006  


In [139]:
# Formateo de la info para generar los datos de salida
focos = list(incendios['WGS84(GPS)'])
timestampFocos = [date+' '+time for date,time in zip(incendios['date'],incendios['time'])]

print(focos)
print(timestampFocos)

['(-71.99229230508983, -37.99943823976221)', '(-72.6755660110673, -37.59965099716092)', '(-72.7246134074418, -38.08247391918)']
['2016-01-14 20:00', '2016-01-14 20:00', '2016-01-14 20:00']


In [241]:
# se excluyen a las interfaz
import numpy as np
import numpy.random as rn
from scipy.stats import truncexpon
from ast import literal_eval as make_tuple
import requests

from datetime import datetime, timedelta

# id >> tipo >> horasDeTrabajo >> latitud >> longitud >> estaActivo >> estaAgrupado;
# horasDeTrabajo -> 0-

recursos = [] # id, tipo, horasTrabajo, lat, lon, activo, agrupado

contador_diferencia = [] #un calculo rápido para la diferencia en distancias
velocidad = [] #un calculo rápido para la velocidad en ruta
conversion = 11.58 #m/s

activo_pr = 0.95
agrupado_pr = 1
preparacion_rec_aereos = [15*60,20*60] # Entre 15 a 20 min

#por foco
# focos =['(-72.95020583491079, -37.73029279769046)', '(-73.01273041486445, -38.16781738573914)', '(-73.32945304026725, -38.21793985248813)']
# timestampFocos = ['12/22/2015T8:00', '12/22/2015T8:00', '12/22/2015T8:00']

for index,item in recursosDF.iterrows():
    item['Clave']
    point_32518 = Point(item['UTM (X)'],item['UTM (Y)'])
    point_wgs = transform(project_3,point_32518) # de epsg:32518 a WGS84 epsg:4326
    activo = 1 if (rn.rand()<activo_pr) else 0
    horasDeTrabajo = truncexpon.rvs(8) if (activo) else 0.0
    asignado = -1
    agrupado = 1 if (rn.rand()<agrupado_pr) else 0
    etas = []
    for timestampFoco,foco in zip(timestampFocos, focos):
        foco_wgs = Point(*make_tuple(foco))
        foco_32518 = transform(project_6, foco_wgs)
        timestamp = datetime.strptime(timestampFoco,'%Y-%m-%d %H:%M')
        
        #api:url con epsg:4326
        url = 'http://router.project-osrm.org/route/v1/driving/'+str(point_wgs.x)+','\
                +str(point_wgs.y)+';'+str(foco_wgs.x)+','+str(foco_wgs.y)+'?overview=false'
        res = requests.get(url)
        if item['Tipo'] in velocidades:
            duracion = (distance((point_wgs.x,point_wgs.y), (foco_wgs.x,foco_wgs.y))/velocidades[item['Tipo']])*3600
            duracion += random.randint(*preparacion_rec_aereos)
        else:
            if res.status_code == 200:
                if res.json()['code']=='Ok':
                    distancia = res.json()['routes'][0]['distance'] #en metros
                    duracion = res.json()['routes'][0]['duration'] #en segundos
                    contador_diferencia.append(distancia-point_32518.distance(foco_32518))
                    velocidad.append(distancia/duracion) # en m/s
            else:
                print('Error de Request.')
                if velocidad:
                    conversion = np.mean(velocidad)
                duracion = point_32518.distance(foco_32518) / conversion
        timestamp = timestamp+timedelta(seconds=duracion)
        etas.append(timestamp.strftime("%Y-%m-%dT%H:%M"))
    #print([item['Clave'],item['Tipo'],horasDeTrabajo,point_wgs.x,point_wgs.y,activo,agrupado,*etas])
    recursos.append([item['Clave'],item['Tipo'],horasDeTrabajo,point_wgs.x,point_wgs.y,activo,asignado,agrupado,*etas])

del point_32518, point_wgs, horasDeTrabajo, activo, agrupado, res, url, foco_wgs, foco_32518

In [242]:
# LATITUD_FOCO_1 LONGITUD_FOCO_1
#  TIMESTAMP_INICIO VALOR_RODAL_USD_POR_HECTAREA MODELO_COMBUSTIBLE DISTANCIA_A_CIUDAD_MAS_CERCANA CANTIDAD_METROS_CONSTRUIDA
vpl_val = [0.01888,0.016027,0.010235,0.00869,0.001009,0.007603,0.008147,0.001672,0.004886,0.010321,0.009234,0.001787,0.004342,0.002249,0.001441,0.000979,0.001556,0.002365,0.013174,0.005973,0.002481,0.002712,0.006516,0.003255,0.002596,0.009777,0.005429,0.003799,0.001325,0.002134,0.001903]
vpl_key = ['PCH1','PCH2','PCH3','PCH4','PCH5','MT01','MT02','MT03','MT04','MT05','MT06','MT07','MT08','BN01','BN02','BN03','BN04','BN05','PL01','PL02','PL03','PL04','PL05','PL06','PL07','PL08','PL09','PL10','PL11','DX01','DX02']

modelo_mapa = {}

for val,key in zip(vpl_val,vpl_key):
    modelo_mapa[val] = key

def printIncendios(printer=print):
    printer(numeroIncendios)
    
    for i,incendio in incendios.iterrows():
        H = incendio['H (%)']
        V = incendio['V (m/s)']
        D = 0.0
        T = incendio['T (C)']
        P = incendio['pendiente']
        VPL = incendio['VPL']
        time_stamp = incendio['date']+"T"+incendio['time']
        rodal = 9000
        modelo_combustile = modelo_mapa[incendio['VPL']]
        distancia_ciudad = 50000
        metros_construido = 0
        printer("".join(incendio['WGS84(GPS)'][1:-1].split(','))) ### ODIO PANDAS
        printer(H, V, P, T, P, VPL, time_stamp, rodal, modelo_combustile, distancia_ciudad, metros_construido)

In [243]:
def printRecursos(printer=print):
    printer(len(recursos))
    for r in recursos:
        string = str()
        for item in r:
            string += str(item) + ' '
        printer(string)

In [244]:
def printMatriz(printer=print):
    for incendio in range(len(focos)):
        for r in recursos:
            printer(r[0],incendio+1, 1)

In [245]:
str_rendimientos = '''176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667
176.6666667 176.6666667 176.6666667 530 441.6666667 441.6666667 441.6666667'''
def printRendimientos(printer=print):
    printer(str_rendimientos)

In [246]:
str_costos = '''4988 14
30 14
37 21
103 83
4204 4204
53663 53663
4204 4204'''
def printCostos(printer=print):
    printer(str_costos)

In [247]:
#Output practicamente listo. Solo Faltaria el timestamp del momento del llamado al solver
def outputData(printer=print):
    printer(incendios.iloc[0]['date']+'T'+incendios.iloc[0]['time'])
    printIncendios(printer)
    printRecursos(printer)
    printRendimientos(printer)
    printCostos(printer)
    printMatriz(printer)
    
def outputToFile(filename):
    with open(filename, 'w') as f:
        def wrapper(*argv):
            for arg in argv:
                f.write(str(arg))
            f.write('\n')
        
        outputData(wrapper)

In [248]:
filename = 'Realista3Incendios1.txt'

# outputToFile(filename)
outputData()

2016-01-14T20:00
3
-71.99229230508983 -37.99943823976221
52.3576 7.861789999999999 25.5556 14.1205 25.5556 0.006516 2016-01-14T20:00 9000 PL05 50000 0
-72.6755660110673 -37.59965099716092
54.5716 5.7845 30.0 12.1522 30.0 0.002596 2016-01-14T20:00 9000 PL07 50000 0
-72.7246134074418 -38.08247391918
56.8467 5.34006 4.44444 13.6859 4.44444 0.013174 2016-01-14T20:00 9000 PL01 50000 0
48
417 HELITRANSPORTADA 0.5131562347798452 -72.90835039740145 -37.16118976245142 1 -1 1 2016-01-14T20:48 2016-01-14T20:28 2016-01-14T20:26 
415 TERRESTRE 0.9497929752323261 -72.90835039740145 -37.16118976245142 1 -1 1 2016-01-14T23:13 2016-01-14T22:19 2016-01-14T23:49 
612 HELITRANSPORTADA 0.2263557945712022 -72.41893931363812 -37.94166502251093 1 -1 1 2016-01-14T20:31 2016-01-14T20:26 2016-01-14T20:28 
614 TERRESTRE 0.058464100991969796 -72.41893931363812 -37.94166502251093 1 -1 1 2016-01-14T21:32 2016-01-14T21:19 2016-01-14T21:52 
611 CISTERNA 0.49692601795542185 -72.41893931363812 -37.94166502251093 1 -1 1 

In [18]:
import  csv



with open("test_2.txt","w") as f:
    #Falta agregar la info del fuego
    
    #Falta el indicador de cuantos son
    wr = csv.writer(f,delimiter=" ")
    wr.writerows(recursos)
    #Falta la info de las compatibilidades

In [ ]:
#Algunas restricciones de prueba

solo se puede 6:23 AM - 9:21 PM crepusculo

In [1]:
recursos = [[315, 'TERRESTRE', 0.0, -73.39782868981612, -37.803053870247474, 0, 0, 15799.4],
['Z-5', 'CISTERNA', 0.0, -72.68643948191286, -37.79638710806514, 0, 1, 9333.5],
['Z-6', 'CISTERNA', 0.0, -72.68643948191286, -37.79638710806514, 0, 0, 9333.5],
['Z-9', 'CISTERNA', 0.0, -72.68643948191286, -37.79638710806514, 0, 0, 9333.5],
['W-10', 'CHINOOK', 0.0, -72.68643948191286, -37.79638710806514, 0, 0, 9333.5],
[417, 'HELITRANSPORTADA', 0.0, -72.90835039740145, -37.161189762451414, 0, 0, 9996.8],
[415, 'TERRESTRE', 0.0, -72.90835039740145, -37.161189762451414, 0, 0, 9996.8],
[512, 'HELITRANSPORTADA', 0.0, -72.63032918220371, -37.75916561481369, 0, 0, 8884.9],
[511, 'CISTERNA', 0.0, -72.63033878514649, -37.75916346504277, 0, 0, 8884.8],
[413, 'TERRESTRE', 0.0, -72.49661044770887, -37.15008355747122, 0, 0, 6355.4],
[416, 'TERRESTRE', 0.0, -72.61199495823381, -37.40388694147487, 0, 0, 7852.9],
[215, 'TERRESTRE', 0.0, -72.20643821059261, -35.62305261097221, 0, 0, 16127.4],
[318, 'HELITRANSPORTADA', 2.9643033040309907, -72.62421724797369, -37.51333138436266, 1, 0, 8334.1],
[311, 'CISTERNA', 0.0, -72.62421724797369, -37.51333138436266, 0, 0, 8334.1],
[319, 'MECANIZADA', 3.1538585199081606, -72.62421724797369, -37.51333138436266, 1, 0, 8334.1],
['Z-4', 'CISTERNA', 0.0, -72.3250808626428, -35.87715872895046, 0, 0, 14475],
[316, 'TERRESTRE', 0.0, -73.21949520969076, -38.010553847724736, 0, 0, 13312.4],
[613, 'TERRESTRE', 0.3978509584289313, -72.2647738680613, -37.72999929404411, 1, 0, 7477.4],
[517, 'HELITRANSPORTADA', 0.0, -72.73838406720967, -37.79360943397726, 0, 0, 9746.9],
[519, 'MECANIZADA', 0.0, -72.73838406720967, -37.79360943397726, 0, 0, 9746.9],
['Z-2', 'CISTERNA', 0.0, -71.60423365986381, -35.37110546980738, 0, 0, 13460.6],
[711, 'TERRESTRE', 0.0, -72.79477287784924, -38.620831980904754, 0, 0, 14380],
[713, 'CISTERNA', 1.4221412248900738, -72.79477287784924, -38.620831980904754, 1, 0, 14380],
[212, 'HELITRANSPORTADA', 0.0, -71.98062679258524, -35.62360879227027, 0, 0, 14295.8],
[211, 'CISTERNA', 0.0, -71.98062679258524, -35.62360879227027, 0, 0, 14295.8],
[219, 'MECANIZADA', 0.9605523755598311, -71.98062679258524, -35.62360879227027, 1, 0, 14295.8],
[712, 'TERRESTRE', 0.0, -72.63679393187564, -38.99261606368905, 0, 0, 13760.7],
[312, 'TERRESTRE', 1.3282936174400979, -73.05616198813235, -36.904997857001455, 1, 0, 9970.8],
[615, 'TERRESTRE', 0.0, -72.33156929125937, -38.23139027616788, 0, 0, 10036.1],
[213, 'TERRESTRE', 0.0, -71.80504968076885, -35.391663719584734, 0, 0, 14274.6],
[612, 'HELITRANSPORTADA', 0.0, -72.41893931363812, -37.94166502251093, 0, 0, 8664],
[614, 'TERRESTRE', 0.5162282813429611, -72.41893931363812, -37.94166502251093, 1, 0, 8664],
[611, 'CISTERNA', 0.45234807920864634, -72.41893931363812, -37.94166502251093, 1, 0, 8664],
[3110, 'TERRESTRE', 0.0, -72.6926124133748, -37.52302672645459, 0, 1, 8562.7],
[811, 'Interfaz', 0.0, -72.6926124133748, -37.52302672645459, 0, 0, 8562.7],
[812, 'Interfaz', 1.2010761209165481, -72.6926124133748, -37.52302672645459, 1, 0, 8562.7],
[214, 'TERRESTRE', 0.0, -72.3992172251887, -35.483330500418674, 0, 0, 17974.4],
[414, 'TERRESTRE', 0.0, -72.07211742347977, -37.166391252946624, 0, 0, 3737.5],
['Z-1', 'CISTERNA', 0.0, -72.41255045082566, -37.12860903604489, 0, 0, 6025.6],
['W-9', 'CHINOOK', 0.0, -72.4131060920789, -37.1377757242435, 0, 0, 6377.2],
[412, 'HELITRANSPORTADA', 0.0, -72.4131060920789, -37.1377757242435, 0, 0, 6377.2],
[411, 'CISTERNA', 0.0, -72.4131060920789, -37.1377757242435, 0, 0, 6377.2],
[419, 'MECANIZADA', 0.0, -72.4131060920789, -37.1377757242435, 0, 0, 6377.2],
[216, 'TERRESTRE', 0.0, -70.56504004780913, -33.63554607424722, 0, 0, 22967.1],
[514, 'TERRESTRE', 0.0, -73.23005181471346, -38.302774236728204, 0, 0, 15165.6],
[515, 'TERRESTRE', 2.245393146314659, -73.23005181471346, -38.302774236728204, 1, 0, 15165.6],
[516, 'CISTERNA', 0.0, -73.23005181471346, -38.302774236728204, 0, 0, 15165.6],
['Z-3', 'CISTERNA', 0.0, -72.42393942586999, -37.393609175074175, 0, 0, 7026.7],
[314, 'HELITRANSPORTADA', 0.0, -72.42310614377139, -37.392220280988994, 0, 0, 6983.8],
[313, 'CISTERNA', 0.0, -72.42310614377139, -37.392220280988994, 0, 0, 6983.8]]

In [2]:
len(recursos)

50

In [4]:
for i in recursos:
    print(i[0],1,1)

315 1 1
Z-5 1 1
Z-6 1 1
Z-9 1 1
W-10 1 1
417 1 1
415 1 1
512 1 1
511 1 1
413 1 1
416 1 1
215 1 1
318 1 1
311 1 1
319 1 1
Z-4 1 1
316 1 1
613 1 1
517 1 1
519 1 1
Z-2 1 1
711 1 1
713 1 1
212 1 1
211 1 1
219 1 1
712 1 1
312 1 1
615 1 1
213 1 1
612 1 1
614 1 1
611 1 1
3110 1 1
811 1 1
812 1 1
214 1 1
414 1 1
Z-1 1 1
W-9 1 1
412 1 1
411 1 1
419 1 1
216 1 1
514 1 1
515 1 1
516 1 1
Z-3 1 1
314 1 1
313 1 1
